In [ ]:
# storia più su nns convolutive, capsulenetwork, evoluzione, problemi di classificazione





# usare affnist



# prova minst su affnist e verificare risultati con paper
# testare su dataset normale + rotraslato*2
# clustering per training
# risultato si su db normale sia rototraslato
#
# + provare no shearing


# matrice di confusione normalizzata sia su righe che su colonne (tot 3 matrici)

### Import statements

In [1]:
%%capture

import numpy as np
import tensorflow as tf

#mandatory for correct load and save of files
%cd /Users/paolobonomi/Work/Python/CapsNetwork

# for project class
import sys
sys.path.append("/Users/paolobonomi/Work/Python/CapsNetwork/src")

from setup import Setup # set up model and dataset
import perfu # performance function such as confusion matrix etc...
import printer

### Retrieve model and dataset

In [ ]:
train_dataset_type = Setup.d_k[1]
test_dataset_type = Setup.d_k[0]

In [ ]:
setup = Setup(train_cfg=train_dataset_type, test_cfg=test_dataset_type, epochs=10)

model = setup.get_model()
epochs = model.get_epochs()

X_train, y_train = setup.get_train_images()
X_test, y_test = setup.get_test_images()
dataset = setup.get_dataset()
training = setup.get_testing()

In [ ]:
printer.pretty_experiment_overview(setup)

### Perfomance data

In [ ]:
should_create_matrix = True

In [ ]:
if should_create_matrix:
    confusionmatrix_test = perfu.get_confusion_mat(model.predict, testing, "test")
    %store confusionmatrix_test
    
    confusionmatrix_train = perfu.get_confusion_mat(model.predict, dataset, "train")
    %store confusionmatrix_train
else:
    %store -r confusionmatrix_train
    %store -r confusionmatrix_test 
    
acc_train = perfu.get_accuracy(perfu.normalize_matrix( confusionmatrix_train, X_train.shape[0]))
acc_test = perfu.get_accuracy(perfu.normalize_matrix( confusionmatrix_test, X_test.shape[0]))

#### Testing Confusion Table

In [ ]:
values, columns, index = perfu.get_confusion_table(confusionmatrix_test, X_test.shape[0])
printer.print_confusion_tables(values, columns, index)

#### Confusion Matrix

In [ ]:
printer.print_matrix(
    np.around(perfu.normalize_matrix( confusionmatrix_train, X_train.shape[0]), decimals=3),
    'CapsNet AFFNIST (10 epochs) Confusion Matrix on Training Set ('+str(int(X_train.shape[0]/1000))+'k images)',
    'Actual Digit',
    'Predicted Digit',
    perfu.headers,
    perfu.headers)

In [ ]:
printer.print_matrix(
    np.around(perfu.normalize_matrix( confusionmatrix_test, X_test.shape[0]), decimals=3),
    'CapsNet AFFNIST (10 epochs) Confusion Matrix on Testing Set ('+str(int(X_test.shape[0]/1000))+'k images)',
    'Actual Digit',
    'Predicted Digit',
    perfu.headers,
    perfu.headers)

#### Accuracy

In [ ]:
printer.print_accuracy(acc_train, acc_test, X_train.shape[0], X_test.shape[0])

### Error cases

In [ ]:
off = 44
n = 110
idx, pred = perfu.get_error_index(model, X_test[off:off+n], y_test[off:off+n], off)

print(idx)
print(pred)

In [ ]:
for i in range(len(idx)):
    img = idx[i]
    printer.print_image_and_prediction(X_test[img], y_test[img], pred[i], 40 )


### Network Conv1/PrimaryCapsule Output

In [ ]:
printer.print_network(model, X_test[45], y_test[45], 40)

### Network Kernels

In [ ]:
printer.print_fixed_network_params(model)